# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import pandas as pd
import numpy as np
import requests
import xlsxwriter
from scipy import stats
import math

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [3]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'avgTotalVolume': 94589303,
 'calculationPrice': 'iexlasttrade',
 'change': 4.7,
 'changePercent': 0.03067,
 'close': 0,
 'closeSource': 'aofcifli',
 'closeTime': None,
 'companyName': 'Apple Inc',
 'currency': 'USD',
 'delayedPrice': None,
 'delayedPriceTime': None,
 'extendedChange': None,
 'extendedChangePercent': None,
 'extendedPrice': None,
 'extendedPriceTime': None,
 'high': 0,
 'highSource': None,
 'highTime': None,
 'iexAskPrice': 0,
 'iexAskSize': 0,
 'iexBidPrice': 0,
 'iexBidSize': 0,
 'iexClose': 160.076,
 'iexCloseTime': 1659197104509,
 'iexLastUpdated': 1656506018191,
 'iexMarketPercent': 0.018195551268219853,
 'iexOpen': 163.34,
 'iexOpenTime': 1698070127767,
 'iexRealtimePrice': 166.456,
 'iexRealtimeSize': 201,
 'iexVolume': 1810447,
 'lastTradeTime': 1728792810696,
 'latestPrice': 165.922,
 'latestSource': 'IEX Last Trade',
 'latestTime': 'March 16, 2022',
 'latestUpdate': 1716602376287,
 'latestVolume': None,
 'low': 0,
 'lowSource': None,
 'lowTime': None,
 'mark

## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [4]:
price = data['latestPrice']
pe_ratio = data['peRatio']
pe_ratio

27.12

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [6]:
final_dataframe = pd.DataFrame(columns=my_columns)
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,news&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        try:
            final_dataframe = final_dataframe.append(
            pd.Series(
            [
                symbol,
                data[symbol]['quote']['latestPrice'],
                data[symbol]['quote']['peRatio'],
                'N/A'
            ],index=my_columns),
                ignore_index = True
            )
        except:
            pass
        
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,138,35.19,N/A
1,AAL,17.23,-5.54,N/A
2,AAP,211.42,22.51,N/A
3,AAPL,162.813,27.5,N/A
4,ABBV,156.43,25.0,N/A
...,...,...,...,...
497,YUM,123.7,23.3,N/A
498,ZBH,128.04,66.96,N/A
499,ZBRA,414.01,26.56,N/A
500,ZION,70.14,6.91,N/A


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [7]:
final_dataframe.sort_values('Price-to-Earnings Ratio',ascending = False,inplace=True)
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio']>0]
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace=True)
final_dataframe.drop('index',axis=1,inplace=True)
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,NOW,557.4,499.97,N/A
1,VTR,58.79,475.91,N/A
2,DXCM,448.21,289.12,N/A
3,KSU,295.3,283,N/A
4,FTI,7.07,249.18,N/A
5,ALB,192.52,186.1,N/A
6,SBAC,327.13,156.77,N/A
7,FIS,99.84,146.99,N/A
8,CRM,206.96,144.5,N/A
9,EQIX,728.27,133.34,N/A


## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [9]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [10]:
portfolio_input()

Enter the value of your portfolio:10000000


You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [15]:
position_size = float(portfolio_size)/len(final_dataframe.index)
for row in final_dataframe.index:
    final_dataframe.loc[row,'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[row,'Price'])
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,NOW,557.4,499.97,358
1,VTR,58.79,475.91,3401
2,DXCM,448.21,289.12,446
3,KSU,295.3,283,677
4,FTI,7.07,249.18,28288
5,ALB,192.52,186.1,1038
6,SBAC,327.13,156.77,611
7,FIS,99.84,146.99,2003
8,CRM,206.96,144.5,966
9,EQIX,728.27,133.34,274


## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [27]:
symbol = 'AAPL'
batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()

In [28]:
# Price-to-earnings ratio
pe_ration = data[symbol]['quote']['peRatio']
# Price-to-book ratio
pb_ration = data[symbol]['advanced-stats']['priceToBook']
# Price-to-sales ratio
ps_ratio = data[symbol]['advanced-stats']['priceToSales']

enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
ebitda = data[symbol]['advanced-stats']['EBITDA']

# Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
ev_to_ebitda = enterprise_value/ebitda
gross_profit = data[symbol]['advanced-stats']['grossProfit']
# Enterprise Value divided by Gross Profit (EV/GP)
ev_to_gp = enterprise_value/gross_profit

{'AAPL': {'advanced-stats': {'beta': 1.3197875577226572,
   'totalCash': 63929293620,
   'currentDebt': 153733519841,
   'revenue': 391243986955,
   'grossProfit': 164392780416,
   'totalRevenue': 381281964857,
   'EBITDA': 128760717372,
   'revenuePerShare': 24.29,
   'revenuePerEmployee': 2692309.19,
   'debtToEquity': 5.438943308079931,
   'profitMargin': 0.2733816774157624,
   'enterpriseValue': 2681210444597,
   'enterpriseValueToRevenue': 7.01,
   'priceToSales': 6.83,
   'priceToBook': 35.84,
   'forwardPERatio': 25.457470040437386,
   'pegRatio': 0.4084471780078059,
   'peHigh': 28.197109944839276,
   'peLow': 14.746945450205926,
   'week52highDate': '2021-12-22',
   'week52lowDate': '2021-03-28',
   'putCallRatio': 0.5168741300209312,
   'companyName': 'Apple Inc',
   'marketcap': 2651112241563,
   'week52high': 186.51,
   'week52low': 118.72,
   'week52highSplitAdjustOnly': 188.44,
   'week52highDateSplitAdjustOnly': '2021-12-20',
   'week52lowSplitAdjustOnly': 124.07,
   'we

Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [52]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'Price-To-Earnings Ratio',
    'PE Percentile',
    'Price-To-Book Ratio',
    'PB Percentile',
    'Price-To-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV-To-GP',
    'EV-To-GP Percentile',
    'RV-Score'
]

In [55]:
rv_dataframe = pd.DataFrame(columns=rv_columns)
for symbol_string in symbol_strings:
    batch_api_all_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_all_url).json()
    for symbol in symbol_string.split(','):
        try:
            enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
            gross_profit = data[symbol]['advanced-stats']['grossProfit']
            ebitda = data[symbol]['advanced-stats']['EBITDA']

            try:
                ev_to_ebitda = enterprise_value/ebitda
            except:
                ev_to_ebitda = np.NAN

            try:
                ev_to_gross = enterprise_value/gross_profit
            except:
                ev_to_gross = np.NAN

            rv_dataframe = rv_dataframe.append(
            pd.Series([
                symbol,
                data[symbol]['quote']['latestPrice'],
                'N/A',
                data[symbol]['quote']['peRatio'],
                'N/A',
                data[symbol]['advanced-stats']['priceToBook'],
                'N/A',
                data[symbol]['advanced-stats']['priceToSales'],
                'N/A',
                ev_to_ebitda,
                'N/A',
                ev_to_gross,
                'N/A',
                'N/A'
            ],index=rv_columns),
            ignore_index = True
            )
        except:
            pass
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-To-Earnings Ratio,PE Percentile,Price-To-Book Ratio,PB Percentile,Price-To-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV-To-GP,EV-To-GP Percentile,RV-Score
0,A,139,N/A,35.42,N/A,7.61,N/A,6.35,N/A,24.165227,N/A,11.751948,N/A,N/A
1,AAL,16.79,N/A,-5.44,N/A,-1.44,N/A,0.349,N/A,-11.793187,N/A,1.175818,N/A,N/A
2,AAP,210.53,N/A,22.05,N/A,4.15,N/A,1.15,N/A,11.539132,N/A,2.558891,N/A,N/A
3,AAPL,161.57,N/A,27.51,N/A,35.26,N/A,6.81,N/A,20.501944,N/A,15.770538,N/A,N/A
4,ABBV,162.96,N/A,24.99,N/A,18.33,N/A,4.97,N/A,12.083200,N/A,8.651341,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
497,YUM,122.62,N/A,23.7,N/A,-4.03,N/A,5.07,N/A,19.132187,N/A,8.964449,N/A,N/A
498,ZBH,123.4,N/A,63.87,N/A,2.01,N/A,3.32,N/A,18.670535,N/A,5.830799,N/A,N/A
499,ZBRA,405.09,N/A,25.95,N/A,7.01,N/A,3.75,N/A,18.895272,N/A,8.128958,N/A,N/A
500,ZION,70.86,N/A,6.73,N/A,1.43,N/A,2.83,N/A,4.971865,N/A,2.565096,N/A,N/A


## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

In [57]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-To-Earnings Ratio,PE Percentile,Price-To-Book Ratio,PB Percentile,Price-To-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV-To-GP,EV-To-GP Percentile,RV-Score
28,ALXN,186.5,N/A,59.57,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
40,AON,322.82,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
71,BRK.B,338.08,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
118,CTL,12,N/A,10.03,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
136,DISCK,26.27,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
165,ETFC,50.52,N/A,14.72,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
186,FLIR,57.98,N/A,33.5,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
190,FOX,37.8,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
193,FRT,122.5,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
204,GOOG,2697.47,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A


Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

In [65]:
for column in [
    'Price-To-Earnings Ratio',
    'Price-To-Book Ratio',
    'Price-To-Sales Ratio',
    'EV/EBITDA',
    'EV-To-GP']:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(),inplace=True) 

Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [66]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-To-Earnings Ratio,PE Percentile,Price-To-Book Ratio,PB Percentile,Price-To-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV-To-GP,EV-To-GP Percentile,RV-Score


## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [69]:
metrics = {
    'Price-To-Earnings Ratio':'PE Percentile',
    'Price-To-Book Ratio':'PB Percentile',
    'Price-To-Sales Ratio':'PS Percentile',
    'EV/EBITDA':'EV/EBITDA Percentile',
    'EV-To-GP':'EV-To-GP Percentile'
}

for metric in metrics.keys():
    for row in rv_dataframe.index:
        rv_dataframe.loc[row,metrics[metric]] = stats.percentileofscore(rv_dataframe[metric],rv_dataframe.loc[row,metric])
        
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-To-Earnings Ratio,PE Percentile,Price-To-Book Ratio,PB Percentile,Price-To-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV-To-GP,EV-To-GP Percentile,RV-Score
0,A,139,N/A,35.42,75.896414,7.61,78.884462,6.350,81.2749,24.165227,82.47012,11.751948,79.282869,N/A
1,AAL,16.79,N/A,-5.44,3.984064,-1.44,5.776892,0.349,1.792829,-11.793187,4.780876,1.175818,3.585657,N/A
2,AAP,210.53,N/A,22.05,46.015936,4.15,61.155378,1.150,15.139442,11.539132,37.450199,2.558891,12.749004,N/A
3,AAPL,161.57,N/A,27.51,61.155378,35.26,97.011952,6.810,83.167331,20.501944,73.904382,15.770538,89.442231,N/A
4,ABBV,162.96,N/A,24.99,54.98008,18.33,93.227092,4.970,73.306773,12.083200,39.442231,8.651341,63.545817,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
497,YUM,122.62,N/A,23.70,51.394422,-4.03,5.378486,5.070,74.003984,19.132187,69.521912,8.964449,67.131474,N/A
498,ZBH,123.4,N/A,63.87,93.426295,2.01,27.788845,3.320,52.091633,18.670535,67.928287,5.830799,38.446215,N/A
499,ZBRA,405.09,N/A,25.95,58.366534,7.01,75.896414,3.750,57.370518,18.895272,68.525896,8.128958,56.175299,N/A
500,ZION,70.86,N/A,6.73,9.063745,1.43,15.537849,2.830,45.119522,4.971865,9.960159,2.565096,12.948207,N/A


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [75]:
from statistics import mean
for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row,metrics[metric]])
    rv_dataframe.loc[row,'RV-Score'] = mean(value_percentiles)
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-To-Earnings Ratio,PE Percentile,Price-To-Book Ratio,PB Percentile,Price-To-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV-To-GP,EV-To-GP Percentile,RV-Score
0,A,139,N/A,35.42,75.896414,7.61,78.884462,6.350,81.2749,24.165227,82.47012,11.751948,79.282869,79.561753
1,AAL,16.79,N/A,-5.44,3.984064,-1.44,5.776892,0.349,1.792829,-11.793187,4.780876,1.175818,3.585657,3.984064
2,AAP,210.53,N/A,22.05,46.015936,4.15,61.155378,1.150,15.139442,11.539132,37.450199,2.558891,12.749004,34.501992
3,AAPL,161.57,N/A,27.51,61.155378,35.26,97.011952,6.810,83.167331,20.501944,73.904382,15.770538,89.442231,80.936255
4,ABBV,162.96,N/A,24.99,54.98008,18.33,93.227092,4.970,73.306773,12.083200,39.442231,8.651341,63.545817,64.900398
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
497,YUM,122.62,N/A,23.70,51.394422,-4.03,5.378486,5.070,74.003984,19.132187,69.521912,8.964449,67.131474,53.486056
498,ZBH,123.4,N/A,63.87,93.426295,2.01,27.788845,3.320,52.091633,18.670535,67.928287,5.830799,38.446215,55.936255
499,ZBRA,405.09,N/A,25.95,58.366534,7.01,75.896414,3.750,57.370518,18.895272,68.525896,8.128958,56.175299,63.266932
500,ZION,70.86,N/A,6.73,9.063745,1.43,15.537849,2.830,45.119522,4.971865,9.960159,2.565096,12.948207,18.525896


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [79]:
rv_dataframe.sort_values('RV-Score',ascending=True,inplace=True)
rv_dataframe = rv_dataframe[:50]
rv_dataframe.reset_index(drop=True,inplace=True)
rv_dataframe

/Users/chayanrellan/opt/anaconda3/lib/python3.9/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


,Ticker,Price,Number of Shares to Buy,Price-To-Earnings Ratio,PE Percentile,Price-To-Book Ratio,PB Percentile,Price-To-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV-To-GP,EV-To-GP Percentile,RV-Score
0,AAL,16.79,N/A,-5.44,3.984064,-1.4400,5.776892,0.3490,1.792829,-11.793187,4.780876,1.175818,3.585657,3.984064
1,UNM,30.36,N/A,7.44,10.557769,0.5101,5.976096,0.4858,3.784861,3.310628,6.175299,0.461221,0.199203,5.338645
2,PRU,119.61,N/A,6.12,7.569721,0.6962,6.972112,0.5850,5.976096,3.982569,7.768924,0.587220,0.796813,5.816733
3,AIG,61.59,N/A,5.72,6.573705,0.7404,7.171315,0.9556,11.354582,3.674277,6.972112,0.960599,2.390438,6.89243
4,GM,44.86,N/A,6.82,9.561753,1.0500,9.462151,0.4900,3.984064,2.205572,5.776892,1.853436,8.964143,7.549801
5,LNC,66.42,N/A,8.96,12.549801,0.5466,6.175299,0.5574,5.179283,6.171297,14.741036,0.563402,0.398406,7.808765
6,MET,70.45,N/A,9.30,13.14741,0.8082,7.370518,0.7628,8.167331,5.244243,11.354582,0.776145,1.195219,8.247012
7,F,16.77,N/A,3.90,5.378486,1.4000,15.139442,0.4777,3.585657,4.062418,8.366534,2.196782,10.756972,8.645418
8,C,57.1,N/A,5.89,6.972112,0.5989,6.374502,1.4200,21.314741,3.432993,6.573705,1.297394,5.179283,9.282869
9,KSS,64.45,N/A,10.00,15.139442,1.5500,18.426295,0.4046,2.589641,4.028163,7.968127,1.255244,4.183267,9.661355


## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [81]:
portfolio_input() 

Enter the value of your portfolio:2500000


In [87]:
position_size = float(portfolio_size)/len(rv_dataframe.index)
for row in rv_dataframe.index:
    rv_dataframe.loc[row,'Number of Shares to Buy'] = math.floor(position_size/rv_dataframe.loc[row,'Price'])
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-To-Earnings Ratio,PE Percentile,Price-To-Book Ratio,PB Percentile,Price-To-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV-To-GP,EV-To-GP Percentile,RV-Score
0,AAL,16.79,2977,-5.44,3.984064,-1.4400,5.776892,0.3490,1.792829,-11.793187,4.780876,1.175818,3.585657,3.984064
1,UNM,30.36,1646,7.44,10.557769,0.5101,5.976096,0.4858,3.784861,3.310628,6.175299,0.461221,0.199203,5.338645
2,PRU,119.61,418,6.12,7.569721,0.6962,6.972112,0.5850,5.976096,3.982569,7.768924,0.587220,0.796813,5.816733
3,AIG,61.59,811,5.72,6.573705,0.7404,7.171315,0.9556,11.354582,3.674277,6.972112,0.960599,2.390438,6.89243
4,GM,44.86,1114,6.82,9.561753,1.0500,9.462151,0.4900,3.984064,2.205572,5.776892,1.853436,8.964143,7.549801
5,LNC,66.42,752,8.96,12.549801,0.5466,6.175299,0.5574,5.179283,6.171297,14.741036,0.563402,0.398406,7.808765
6,MET,70.45,709,9.30,13.14741,0.8082,7.370518,0.7628,8.167331,5.244243,11.354582,0.776145,1.195219,8.247012
7,F,16.77,2981,3.90,5.378486,1.4000,15.139442,0.4777,3.585657,4.062418,8.366534,2.196782,10.756972,8.645418
8,C,57.1,875,5.89,6.972112,0.5989,6.374502,1.4200,21.314741,3.432993,6.573705,1.297394,5.179283,9.282869
9,KSS,64.45,775,10.00,15.139442,1.5500,18.426295,0.4046,2.589641,4.028163,7.968127,1.255244,4.183267,9.661355


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

## Saving Our Excel Output
As before, saving our Excel output is very easy: